In [1]:
import csv
from pyspark import SparkContext
sc=SparkContext()

24/12/13 17:50:21 WARN Utils: Your hostname, Zhijie resolves to a loopback address: 127.0.1.1; using 10.27.109.93 instead (on interface wlo1)
24/12/13 17:50:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 17:50:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/13 17:50:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/13 17:50:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
data1 = sc.textFile('Phones_accelerometer.csv', 8)
data2 = sc.textFile('Phones_gyroscope.csv', 8)
data3 = sc.textFile('Watch_accelerometer.csv', 8)
data4 = sc.textFile('Watch_gyroscope.csv', 8)

In [3]:
def configure_data(line):
    datas = line.split(',')
    if datas[9] == 'null':
        return None, None

    x = float(datas[3])
    y = float(datas[4])
    z = float(datas[5])
    key = (datas[6], datas[7], datas[9])
    value = (
        x, y, z,
        x**2, y**2, z**2,
        x, y, z,
        x, y, z,
        1
    )

    return key, value

def filter_data(data):
    key, _ = data
    if key is None:
        return False
    return True

def reduce_data(a, b):
    sum_x = a[0] + b[0]
    sum_y = a[1] + b[1]
    sum_z = a[2] + b[2]
    sum_x2 = a[3] + b[3]
    sum_y2 = a[4] + b[4]
    sum_z2 = a[5] + b[5]
    max_x = max(a[6], b[6])
    max_y = max(a[7], b[7])
    max_z = max(a[8], b[8])
    min_x = min(a[9], b[9])
    min_y = min(a[10], b[10])
    min_z = min(a[11], b[11])
    count = a[12] + b[12]
    return (
        sum_x, sum_y, sum_z,
        sum_x2, sum_y2, sum_z2,
        max_x, max_y, max_z,
        min_x, min_y, min_z,
        count
    )

def calculate_data(data):
    key, value = data
    sum_x, sum_y, sum_z, sum_x2, sum_y2, sum_z2, max_x, max_y, max_z, min_x, min_y, min_z, count = value
    mean_x = sum_x / count
    mean_y = sum_y / count
    mean_z = sum_z / count
    std_x = (sum_x2 / count - mean_x**2)**0.5
    std_y = (sum_y2 / count - mean_y**2)**0.5
    std_z = (sum_z2 / count - mean_z**2)**0.5
    return key, (mean_x, mean_y, mean_z, std_x, std_y, std_z, max_x, max_y, max_z, min_x, min_y, min_z)

def flat_data(data):
    key, value = data
    return key[0], key[1], key[2], value[0], value[1], value[2], value[3], value[4], value[5], value[6], value[7], value[8], value[9], value[10], value[11]

def simplify_data(data):
    key, (value1, value2) = data
    combined_values = value1 + value2
    return key, combined_values

In [4]:
r_data1 = data1.map(configure_data).filter(filter_data).reduceByKey(reduce_data).map(calculate_data)
r_data2 = data2.map(configure_data).filter(filter_data).reduceByKey(reduce_data).map(calculate_data)
r_data3 = data3.map(configure_data).filter(filter_data).reduceByKey(reduce_data).map(calculate_data)
r_data4 = data4.map(configure_data).filter(filter_data).reduceByKey(reduce_data).map(calculate_data)

phones = r_data1.join(r_data2).map(simplify_data)
watch = r_data3.join(r_data4).map(simplify_data)

result = phones.union(watch).collect()

In [5]:
for key, value in result:
    print(key, value)

('a', 'nexus4', 'stand') (-6.026499950569997, 0.9334959509016004, 8.013646013120006, 0.184551746735394, 0.24043415892743478, 0.17599985821395306, -5.5202026, 1.9472808999999998, 8.638794, -7.0448303, -0.84251404, 7.149872, 0.0015888519490950034, 0.0010094604656469997, 0.00044218442934899877, 0.04277492705495176, 0.028613015987344483, 0.045941044056574176, 0.6321869000000001, 0.34971620000000003, 0.44873047, -0.16569519, -0.15550232, -0.6001586999999999)
('a', 'gear', 'stand') (-9.289753201219128, -3.137156524200753, -1.066286300996938, 0.41370414741585154, 1.0628532344784418, 0.6173240991928323, -0.5650316, -0.5781997, 1.01574, -12.600683, -11.08276, -2.2625206, 0.022146662344973242, -0.03286732380555579, -0.07143672518383197, 0.07314944656109791, 0.05570243854351144, 0.050177495900925566, 0.81039995, 0.35446674, 1.1475562, -2.0383835, -0.55287224, -1.2319783)
('a', 'gear', 'sit') (-7.604519709183905, -5.5283410786768235, 2.6383265126877022, 0.1855523178504538, 0.27427094510454214, 0.3

In [6]:
output_file = "output.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    header = [
        "Key1", "Key2", "Key3",
        "Value1", "Value2", "Value3", "Value4", "Value5", "Value6",
        "Value7", "Value8", "Value9", "Value10", "Value11", "Value12",
        "Value13", "Value14", "Value15", "Value16", "Value17", "Value18",
        "Value19", "Value20", "Value21", "Value22", "Value23", "Value24"
    ]
    writer.writerow(header)

    # Write the data rows
    for key, value in result:
        writer.writerow(key + value)

print(f"Data saved to {output_file}")

Data saved to output.csv


In [7]:
def read_file(input_file):
    data = []
    with open(input_file, mode="r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            keys = tuple(row[:3])
            values = tuple(map(float, row[3:]))
            data.append((keys, values))
    return data

data = read_file(output_file)

for key, value in data:
    print(key, value)

('a', 'nexus4', 'stand') (-6.026499950569997, 0.9334959509016004, 8.013646013120006, 0.184551746735394, 0.24043415892743478, 0.17599985821395306, -5.5202026, 1.9472808999999998, 8.638794, -7.0448303, -0.84251404, 7.149872, 0.0015888519490950034, 0.0010094604656469997, 0.00044218442934899877, 0.04277492705495176, 0.028613015987344483, 0.045941044056574176, 0.6321869000000001, 0.34971620000000003, 0.44873047, -0.16569519, -0.15550232, -0.6001586999999999)
('a', 'gear', 'stand') (-9.289753201219128, -3.137156524200753, -1.066286300996938, 0.41370414741585154, 1.0628532344784418, 0.6173240991928323, -0.5650316, -0.5781997, 1.01574, -12.600683, -11.08276, -2.2625206, 0.022146662344973242, -0.03286732380555579, -0.07143672518383197, 0.07314944656109791, 0.05570243854351144, 0.050177495900925566, 0.81039995, 0.35446674, 1.1475562, -2.0383835, -0.55287224, -1.2319783)
('a', 'gear', 'sit') (-7.604519709183905, -5.5283410786768235, 2.6383265126877022, 0.1855523178504538, 0.27427094510454214, 0.3

24/12/13 17:50:33 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
